# Case 1 - Load Factor Prediction

In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)  # suppress scientific notation
import datetime
from sklearn.preprocessing import LabelEncoder
import scipy . linalg as lng

In [2]:
# import original xlsx
df = pd.read_excel('Realized Schedule 20210101-20220208.xlsx')

In [3]:
df

,ScheduleTime,Airline,FlightNumber,Destination,AircraftType,FlightType,Sector,SeatCapacity,LoadFactor
0,2021-01-01 06:35:00,IA,874,DEN,73W,J,US,142,0.408451
1,2021-01-01 10:35:00,JZ,818,YHM,AT7,J,CA,74,0.189189
2,2021-01-01 12:05:00,IA,876,DEN,73W,J,US,142,0.570423
3,2021-01-01 13:20:00,CN,514,EST,AT7,J,US,72,0.333333
4,2021-01-01 14:20:00,LJ,3140,DEN,32A,J,US,186,0.204301
...,...,...,...,...,...,...,...,...,...
36765,2022-02-08 18:30:00,VW,986,YYZ,320,J,CA,180,0.522222
36766,2022-02-08 19:30:00,LJ,579,YQU,319,J,CA,156,0.532051
36767,2022-02-08 19:40:00,LJ,506,YUL,319,J,CA,156,0.602564
36768,2022-02-08 19:05:00,CL,2708,DAN,73H,J,US,189,0.417989


In [4]:
################################
#Process 'ScheduleTime' Feature#
################################
df.insert(1,'Year', df['ScheduleTime'].dt.year)  # insert a column for year

# calculate minutes passed from the first day of the year (xxxx-01-01 00:00:00)
df.insert(2,'DateReference', pd.to_datetime(df['Year'].astype(str) + '-01-01 00:00:00'))
df.insert(3,'MinutesPassed', 
         ((df['ScheduleTime'] - df['DateReference'])/pd.Timedelta(1, 'm')).fillna(0).astype(int))
df = df.drop('DateReference', 1)

In [5]:
# force types of each column
df['Year'] = df['Year'].astype(int)
df['MinutesPassed'] = df['MinutesPassed'].astype(int)
df['Airline'] = df['Airline'].astype(str)
df['FlightNumber'] = df['FlightNumber'].astype(int)
df['Destination'] = df['Destination'].astype(str)
df['AircraftType'] = df['AircraftType'].astype(str)
df['FlightType'] = df['FlightType'].astype(str)
df['Sector'] = df['Sector'].astype(str)
df['SeatCapacity'] = df['SeatCapacity'].astype(int)
df['LoadFactor'] = df['LoadFactor'].astype(float)

In [6]:
df

,ScheduleTime,Year,MinutesPassed,Airline,FlightNumber,Destination,AircraftType,FlightType,Sector,SeatCapacity,LoadFactor
0,2021-01-01 06:35:00,2021,395,IA,874,DEN,73W,J,US,142,0.408451
1,2021-01-01 10:35:00,2021,635,JZ,818,YHM,AT7,J,CA,74,0.189189
2,2021-01-01 12:05:00,2021,725,IA,876,DEN,73W,J,US,142,0.570423
3,2021-01-01 13:20:00,2021,800,CN,514,EST,AT7,J,US,72,0.333333
4,2021-01-01 14:20:00,2021,860,LJ,3140,DEN,32A,J,US,186,0.204301
...,...,...,...,...,...,...,...,...,...,...,...
36765,2022-02-08 18:30:00,2022,55830,VW,986,YYZ,320,J,CA,180,0.522222
36766,2022-02-08 19:30:00,2022,55890,LJ,579,YQU,319,J,CA,156,0.532051
36767,2022-02-08 19:40:00,2022,55900,LJ,506,YUL,319,J,CA,156,0.602564
36768,2022-02-08 19:05:00,2022,55865,CL,2708,DAN,73H,J,US,189,0.417989


In [7]:
data = df.iloc[:, 1:].values  # dataframe to numpy after pre-processing, discard initial 'ScheduleTime'

################
#Label encoders#
################

# Year
le_year = LabelEncoder()
data[:, 0] = le_year.fit_transform(data[:, 0])
# data[:, 0] = le_year.inverse_transform(data[:, 0].astype(int))

# Airline
le_airline = LabelEncoder()
data[:, 2] = le_airline.fit_transform(data[:, 2])
# data[:, 2] = le_airline.inverse_transform(data[:, 2].astype(int))

# FlightNumber
le_flight_number = LabelEncoder()
data[:, 3] = le_flight_number.fit_transform(data[:, 3])
# data[:, 3] = le_flight_number.inverse_transform(data[:, 3].astype(int))

# Destination 
le_destination = LabelEncoder()
data[:, 4] = le_destination.fit_transform(data[:, 4])
# data[:, 4] = le_destination.inverse_transform(data[:, 4].astype(int))

# AircraftType
le_aircraft_type = LabelEncoder()
data[:, 5] = le_aircraft_type.fit_transform(data[:, 5])
# data[:, 5] = le_aircraft_type.inverse_transform(data[:, 5].astype(int))

# FlightType
le_flight_type = LabelEncoder()
data[:, 6] = le_flight_type.fit_transform(data[:, 6])
# data[:, 6] = le_flight_type.inverse_transform(data[:, 6].astype(int))

# Sector
le_sector = LabelEncoder()
data[:, 7] = le_sector.fit_transform(data[:, 7])
# data[:, 7] = le_sector.inverse_transform(data[:, 7].astype(int))

data = data.astype(float)

In [8]:
print(data.shape)
print(data)

(36770, 10)
[[    0.           395.            20.         ...    11.
    142.             0.4084507 ]
 [    0.           635.            28.         ...     1.
     74.             0.18918919]
 [    0.           725.            20.         ...    11.
    142.             0.57042254]
 ...
 [    1.         55900.            35.         ...     1.
    156.             0.6025641 ]
 [    1.         55865.             4.         ...    11.
    189.             0.41798942]
 [    1.         55955.             4.         ...    11.
    189.             0.4973545 ]]


In [12]:
# calculate the mean and standard deviation of two ratio features
# seat capacity
seat_capacity_mean = np.mean(data[:, -2])
seat_capacity_std = np.std(data[:, -2])
print('The mean of seat capacity is {}'.format(seat_capacity_mean))
print('The standard deviation of seat capacity is {}'.format(seat_capacity_std))
# load factor
load_factor_mean = np.mean(data[:, -1])
load_factor_std = np.std(data[:, -1])
print('The mean of load factor is {}'.format(load_factor_mean))
print('The standard deviation of load factor is {}'.format(load_factor_std))

The mean of seat capacity is 155.28550448735382
The standard deviation of seat capacity is 58.850999210500184
The mean of load factor is 0.531752306575933
The standard deviation of load factor is 0.2721886897792685


In [13]:
y = data[:, -1]  # features 
X = data[:, :-1]  # target

## Ordinary Least Squares (OLS) - baseline

In [14]:
beta_ols, _, _, _ = lng.lstsq(X, y)
yhat_ols = np.sum(beta_ols*X, axis=1)
MSE_ols = np.sum((y - yhat_ols)**2)/np.size(y)
print('MSE for OLS regression: {}'.format(MSE_ols))

MSE for OLS regression: 0.0673489951120419
